In [1]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

import time
from time import sleep

import numpy as np

from datetime import datetime
from tqdm.notebook import tqdm
   
from echo_teleoperation import Echo
from ur_rtde import UR3Teleop

from realsense import RealSenseCamera
from data_utils import DataCollector
import cv2

Usage for dual arm teleoperation

In [2]:
if __name__ == "__main__":
    device = Echo()
    # If USB 3.0 capture_frequency=30, width=640, height=480
    # If USB 3.2 capture_frequency=30, width=1280, height=720
    camera = RealSenseCamera(capture_frequency=30, width=1280, height=720)
    data_collector = DataCollector(dataset_dir="dataset")

Device found on port: COM14
Port COM14 opened successfully


In [3]:
left_ip = "192.168.1.111"   # IP address for the left arm
right_ip = "192.168.1.110"  # IP address for the right arm

left_base_pose = np.array(
    [
        0.0002879959065467119,
        -1.5719226042376917,
        -1.5715907255755823,
        3.1422481536865234,
        -1.5707929770099085,
        512.0812350745701,
    ]
)

right_base_pose = np.array(
    [
        -0.001450840626851857,
        -1.5725587050067347,
        1.5713553428649902,
        0.0008134841918945312,
        1.5712484121322632,
        9.42502719560732,
    ]
)

# Create teleoperation instances for both arms
left_arm = UR3Teleop(ip=left_ip, base_pose=left_base_pose, lookahead_time=0.1, gain=200)
right_arm = UR3Teleop(ip=right_ip, base_pose=right_base_pose, lookahead_time=0.1, gain=200)

In [4]:
# Move each arm to its respective base pose
left_arm.move_to_base_pose()
right_arm.move_to_base_pose()

In [5]:
def provide_teleoperation(
    left_arm,
    right_arm,
    left_base_pose,
    right_base_pose,
    data_from_echo,
    camera,
    data_collector=None,
    csv_file_path=None,
    img_folder_path=None,
    start_dataset_collection=0
):
    """
    This function performs teleoperation by processing the received robot pose data
    and then sending new target positions to both robot arms. Optionally, if dataset
    collection is enabled (start_dataset_collection flag is True), it captures a camera frame
    and logs the sample using the data_collector.
    """
    # If no data is received, return None.
    if data_from_echo is None:
        return None
    else:
        # Unpack the received data:
        # Index 0: Left arm joint angles (in radians).
        left_arm_position = data_from_echo[0]
        # Index 2: Left gripper position.
        left_gripper_position = data_from_echo[2]

        # Index 1: Right arm joint angles (in radians).
        right_arm_position = data_from_echo[1]
        # Index 3: Right gripper position.
        right_gripper_position = data_from_echo[3]

        # Index 6: Sensitivity mode for teleoperation control.
        sensitivity_mode = data_from_echo[6]

        # Index 5: Flag or counter to trigger dataset collection.
        # start_dataset_collection = data_from_echo[5]

        # If dataset collection is enabled, capture a camera frame.
        if start_dataset_collection:
            frame = camera.get_frame()

        # Adjust target arm positions based on the sensitivity mode.
        if sensitivity_mode == 0:
            # Default sensitivity: use the raw positions.
            left_arm_new_position = left_base_pose + left_arm_position*1.25
            right_arm_new_position = right_base_pose + right_arm_position * 1.25
        elif sensitivity_mode == 1:
            # Medium sensitivity: scale down the movements slightly.
            left_arm_new_position = left_base_pose + left_arm_position / 1.25
            right_arm_new_position = right_base_pose + right_arm_position / 1.25
        else:
            # High sensitivity: further reduce the movements.
            left_arm_new_position = left_base_pose + left_arm_position / 1.75
            right_arm_new_position = right_base_pose + right_arm_position / 1.75

        # Command the left arm to move to the new pose with the updated gripper position.
        left_arm.move_to_pose(
            joints_positions=left_arm_new_position,
            gripper_position=left_gripper_position,
        )

        # Command the right arm to move to the new pose with the updated gripper position.
        right_arm.move_to_pose(
            joints_positions=right_arm_new_position,
            gripper_position=right_gripper_position,
        )

        # If dataset collection is enabled, add a new sample to the dataset.
        if start_dataset_collection:
            data_collector.add_sample(
                left_arm_new_position,
                right_arm_new_position,
                left_gripper_position,
                right_gripper_position,
                frame,
                csv_file_path,
                img_folder_path,
            )

In [6]:
# Main loop for teleoperation and dataset collection.
# The loop continuously reads data from the device and processes it.
for i in tqdm(range(20000)):
    data_from_echo = device.read_pose_rad(dof_count=7, read_force_sensor=True)
    if data_from_echo is None:
        continue

    # Get the flag that determines whether to start dataset collection.
    start_dataset_collection = data_from_echo[5]

    # If dataset collection is not active (flag is 0), operate the robot normally.
    if not start_dataset_collection:
        provide_teleoperation(
            left_arm,
            
            right_arm,
            left_base_pose,
            right_base_pose,
            data_from_echo,
            camera,
            data_collector,
        )
    else:
        # If dataset collection is active, initialize a new trajectory for saving data.
        csv_file_path, img_folder_path = data_collector.new_trajectory()

        # Inner loop: continue dataset collection until the flag turns false.
        for j in tqdm(range(20000)):
            data_from_echo = device.read_pose_rad(dof_count=7, read_force_sensor=True)
            if data_from_echo is None:
                continue

            # Update the flag based on the current data.
            start_dataset_collection = data_from_echo[5]
            if j%2==0:
                provide_teleoperation(
                    left_arm,
                    right_arm,
                    left_base_pose,
                    right_base_pose,
                    data_from_echo,
                    camera,
                    data_collector,
                    csv_file_path,
                    img_folder_path,
                )
            else:
                provide_teleoperation(
                    left_arm,
                    right_arm,
                    left_base_pose,
                    right_base_pose,
                    data_from_echo,
                    camera,
                    data_collector,
                    csv_file_path,
                    img_folder_path,
                    start_dataset_collection,
                )
            # Continue dataset collection as long as the flag is true.
            if start_dataset_collection:
                continue
            else:
                # Break out of the inner loop when dataset collection is disabled.
                break

  0%|          | 0/20000 [00:00<?, ?it/s]

Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received incomplete data: 0 bytes, expected 42 bytes
Error reading sensor data: Received inco

KeyboardInterrupt: 